In [1]:
# Cell 1: Setup and Imports
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Load environment variables
load_dotenv(project_root / '.env')

# Test API key
assemblyai_key = os.getenv('ASSEMBLYAI_API_KEY')
print(f"AssemblyAI API Key loaded: {'✅' if assemblyai_key else '❌'}")
print(f"Key starts with: {assemblyai_key[:10] if assemblyai_key else 'None'}...")


AssemblyAI API Key loaded: ✅
Key starts with: 972365f41d...


In [2]:
import sqlite3

conn = sqlite3.connect("data/app.db")
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("📊 Tables in app.db:")
for table in tables:
    table_name = table[0]
    
    # Get column info for each table
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    
    print(f"\n🔧 {table_name}:")
    for col in columns:
        print(f"   - {col[1]} ({col[2]})")  # column_name (type)

conn.close()

📊 Tables in app.db:

🔧 conversations:
   - id (INTEGER)
   - title (TEXT)
   - raw_text (TEXT)
   - source (TEXT)
   - word_count (INTEGER)
   - created_at (DATETIME)
   - status (TEXT)

🔧 sqlite_sequence:
   - name ()
   - seq ()

🔧 blog_post_ideas:
   - id (INTEGER)
   - conversation_id (INTEGER)
   - title (TEXT)
   - description (TEXT)
   - usefulness_potential (INTEGER)
   - fitwith_seo_strategy (INTEGER)
   - fitwith_content_strategy (INTEGER)
   - inspiration_potential (INTEGER)
   - collaboration_potential (INTEGER)
   - innovation (INTEGER)
   - difficulty (INTEGER)
   - total_score (INTEGER)
   - sent_to_prod (BOOLEAN)
   - raw_llm_response (TEXT)
   - created_at (DATETIME)

🔧 processing_status:
   - id (INTEGER)
   - conversation_id (INTEGER)
   - stage (TEXT)
   - status (TEXT)
   - error_message (TEXT)
   - started_at (DATETIME)
   - completed_at (DATETIME)


In [6]:
# Cell 2: Import Dependencies
import assemblyai as aai
from langgraph.graph import StateGraph
from typing import TypedDict, Optional, List, Dict  # ← Added List here
import glob
import time
from pathlib import Path  # ← Also added Path here

# Import our database
from database.db_operations import db
from database.models import ConversationCreate

print("✅ All imports successful")

✅ All imports successful


In [5]:
## 1. Pydantic Model for Structured Output

from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum

class SpeakerRole(str, Enum):
    """Possible speaker roles in the conversation"""
    CLIENT = "client"
    INTERVIEWER = "interviewer"

class Speaker(BaseModel):
    """Information about a person speaking in the conversation"""
    name: Optional[str] = Field(default=None, description="Name of the speaker if mentioned")
    role: Optional[SpeakerRole] = Field(default=None, description="Role of the speaker in the conversation")
    company: Optional[str] = Field(default=None, description="Company they work for if mentioned")

class Challenge(BaseModel):
    """A challenge or problem mentioned in the conversation"""
    description: Optional[str] = Field(default=None, description="Description of the challenge")
    impact: Optional[str] = Field(default=None, description="How this challenge affects them")
    urgency: Optional[str] = Field(default=None, description="Low, Medium, or High urgency")

class CurrentSolution(BaseModel):
    """How they currently solve their problems"""
    solution: Optional[str] = Field(default=None, description="What they're currently doing")
    satisfaction_level: Optional[str] = Field(default=None, description="How satisfied they are: Very Satisfied, Satisfied, Neutral, Unsatisfied, Very Unsatisfied")
    limitations: Optional[List[str]] = Field(default=[], description="Limitations of current solution")

class Need(BaseModel):
    """A need identified using psychology frameworks like NVC"""
    need_category: Optional[str] = Field(default=None, description="Category of need (e.g., autonomy, efficiency, security, connection)")
    description: Optional[str] = Field(default=None, description="Specific need description")
    intensity: Optional[str] = Field(default=None, description="Low, Medium, or High intensity")

class ExtractedInsights(BaseModel):
    """Complete structured output from conversation analysis"""
    
    # Speakers
    speakers: Optional[List[Speaker]] = Field(default=[], description="People identified in the conversation")
    
    # What they care about
    core_values: Optional[List[str]] = Field(default=[], description="What this person/company cares about most")
    priorities: Optional[List[str]] = Field(default=[], description="Their current priorities and focus areas")
    
    # Challenges
    primary_challenges: Optional[List[Challenge]] = Field(default=[], description="Main problems they're facing")
    secondary_challenges: Optional[List[Challenge]] = Field(default=[], description="Secondary or related problems")
    
    # Current solutions
    current_solutions: Optional[List[CurrentSolution]] = Field(default=[], description="How they solve problems today")
    
    # Needs analysis
    psychological_needs: Optional[List[Need]] = Field(default=[], description="Underlying needs using NVC or similar frameworks")
 


In [12]:
# Cell 12: Raw Blog Idea Model (Simple)
class RawBlogIdea(BaseModel):
    """Raw blog idea from creative agent"""
    title: str
    description: str
    target_audience: str
    content_angle: str
    business_value: str

print("✅ Simple RawBlogIdea model ready")

✅ Simple RawBlogIdea model ready


In [13]:
# Cell 3: Define LangGraph State
class AudioPipelineState(TypedDict):
    file_path: str
    filename: str
    transcript_text: Optional[str]
    conversation_id: Optional[int]
    extracted_insights: Optional[ExtractedInsights]  
    raw_blog_ideas: Optional[List[RawBlogIdea]]           # Pydantic objects from creative agent
    #scored_blog_ideas: Optional[List[BlogPostIdeaCreate]] # Scored Pydantic objects from analyst
    selected_idea_ids: Optional[List[int]]                # User-selected ideas for DB
    
    # Status & error handling
    status: str
    error: Optional[str]


print("✅ State defined")

✅ State defined


In [8]:
#ad Company Strategy Context
def load_company_strategy_context():
    """Load company strategy and SEO strategy for creative context"""
    
    strategy_context = {}
    
    try:
        # Load company strategy
        company_strategy_path = "../data/processed/company_strategy.mkd"
        with open(company_strategy_path, "r", encoding="utf-8") as f:
            strategy_context["company_strategy"] = f.read()
        print(f"✅ Loaded company strategy ({len(strategy_context['company_strategy'])} chars)")
        
        # Load SEO strategy if exists
        seo_strategy_path = "../data/processed/seo_strategy.mkd"
        if os.path.exists(seo_strategy_path):
            with open(seo_strategy_path, "r", encoding="utf-8") as f:
                strategy_context["seo_strategy"] = f.read()
            print(f"✅ Loaded SEO strategy ({len(strategy_context['seo_strategy'])} chars)")
        else:
            strategy_context["seo_strategy"] = "No specific SEO strategy document found."
            print("⚠️ No SEO strategy document found, using default guidance")
            
    except Exception as e:
        print(f"❌ Error loading strategy documents: {e}")
        strategy_context = {
            "company_strategy": "Strategy document not available",
            "seo_strategy": "SEO strategy document not available"
        }
    
    return strategy_context

# Test loading
strategy_context = load_company_strategy_context()
print(f"📊 Strategy context keys: {list(strategy_context.keys())}")

✅ Loaded company strategy (6555 chars)
✅ Loaded SEO strategy (5566 chars)
📊 Strategy context keys: ['company_strategy', 'seo_strategy']


In [9]:
# Cell : Creative Agent Function
def generate_blog_ideas_from_insights(insights: ExtractedInsights, strategy_context: dict) -> List[Dict]:
    """
    Creative agent that generates blog post ideas from extracted insights
    rooted in Big Kids Automation's business context
    """
    
    creative_prompt = f"""
    You are a creative content strategist for Big Kids Automation, a company that helps businesses implement AI and automation solutions.
    
    COMPANY CONTEXT:
    {strategy_context.get('company_strategy', 'Strategy not available')[:1000]}...
    
    SEO STRATEGY:
    {strategy_context.get('seo_strategy', 'SEO strategy not available')[:500]}...
    
    CONVERSATION INSIGHTS TO WORK FROM:
    
    Speakers: {[f"{s.name} ({s.role}) from {s.company}" for s in insights.speakers] if insights.speakers else "Unknown speakers"}
    
    Core Values: {", ".join(insights.core_values) if insights.core_values else "None identified"}
    
    Priorities: {", ".join(insights.priorities) if insights.priorities else "None identified"}
    
    Primary Challenges:
    {chr(10).join([f"- {c.description} (Impact: {c.impact}, Urgency: {c.urgency})" for c in insights.primary_challenges]) if insights.primary_challenges else "None identified"}
    
    Current Solutions:
    {chr(10).join([f"- {s.solution} (Satisfaction: {s.satisfaction_level})" for s in insights.current_solutions]) if insights.current_solutions else "None identified"}
    
    Psychological Needs:
    {chr(10).join([f"- {n.description} ({n.need_category}, {n.intensity} intensity)" for n in insights.psychological_needs]) if insights.psychological_needs else "None identified"}
    
    TASK:
    Generate 4-5 creative blog post ideas that:
    1. Address the challenges and needs identified in this conversation
    2. Align with Big Kids Automation's mission to help businesses with AI/automation
    3. Provide value to potential clients facing similar challenges
    4. Support our SEO and content marketing strategy
    5. Are actionable and practical, not just theoretical
    
    For each blog post idea, provide:
    - title: Clear, engaging title that includes relevant keywords
    - description: 2-3 sentence description of what the post will cover
    - target_audience: Who this post is primarily for
    - content_angle: The unique angle or approach this post takes
    - business_value: How this post helps our business goals
    
    Return ONLY a JSON array in this exact format:
    [
        {{
            "title": "How AI Invoicing Systems Eliminate Payment Tracking Chaos",
            "description": "A practical guide showing how modern AI-powered invoicing systems solve the common problem of tracking payments across multiple transactions. Includes real case studies and implementation steps.",
            "target_audience": "CTOs and finance directors at growing companies",
            "content_angle": "Problem-solution with real case studies",
            "business_value": "Attracts prospects struggling with financial process automation"
        }},
        ...
    ]
    
    Focus on being creative but grounded in real business value. These ideas should feel inspired by the conversation but applicable to a broader audience.
    """
    
    try:
        # Generate ideas using Claude
        response = llm.invoke(creative_prompt)
        
        # Parse JSON response
        blog_ideas = json.loads(response.content)
        
        print(f"✅ Creative agent generated {len(blog_ideas)} blog ideas")
        return blog_ideas
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error in creative agent: {e}")
        print(f"📝 Raw response: {response.content[:300]}...")
        return []
    except Exception as e:
        print(f"❌ Error in creative agent: {e}")
        return []

print("✅ Creative agent function ready")

✅ Creative agent function ready


In [14]:
# Cell 13: Creative Agent Node (Direct Pydantic)
def creative_agent_node(state: AudioPipelineState) -> AudioPipelineState:
    """Creative agent that generates raw blog ideas as Pydantic objects"""
    
    try:
        print("🎨 Starting creative blog idea generation...")
        
        insights = state.get('extracted_insights')
        if not insights:
            return {**state, "error": "No insights available", "status": "error"}
        
        # Load strategy context
        strategy_context = load_company_strategy_context()
        
        # Generate ideas (returns JSON)
        raw_ideas_json = generate_blog_ideas_from_insights(insights, strategy_context)
        
        # Convert directly to Pydantic objects
        raw_blog_ideas = []
        for idea_json in raw_ideas_json:
            try:
                idea = RawBlogIdea(**idea_json)
                raw_blog_ideas.append(idea)
            except Exception as e:
                print(f"⚠️ Skipping invalid idea: {e}")
        
        if raw_blog_ideas:
            print(f"🎉 Generated {len(raw_blog_ideas)} valid blog ideas")
            return {
                **state,
                "raw_blog_ideas": raw_blog_ideas,  # Direct Pydantic objects
                "status": "raw_ideas_generated"
            }
        else:
            return {**state, "error": "No valid ideas generated", "status": "error"}
            
    except Exception as e:
        print(f"❌ Creative agent error: {e}")
        return {**state, "error": str(e), "status": "error"}

print("✅ Creative agent node (direct Pydantic) ready")

✅ Creative agent node (direct Pydantic) ready


In [6]:
# Cell 4: Test AssemblyAI Connection
# Configure AssemblyAI
aai.settings.api_key = os.getenv('ASSEMBLYAI_API_KEY')

# Test with a simple transcription (we'll use a file from temp folder)
def test_assemblyai_connection():
    """Test if AssemblyAI is working"""
    try:
        # Just test the API key is valid
        transcriber = aai.Transcriber()
        print("✅ AssemblyAI connection successful")
        return True
    except Exception as e:
        print(f"❌ AssemblyAI connection failed: {e}")
        return False

test_assemblyai_connection()

✅ AssemblyAI connection successful


True

In [7]:
# Cell 5: Batch File Discovery and Management
def find_audio_files(temp_folder: Path) -> List[Path]:
    """Find all audio files in temp folder"""
    audio_extensions = ['*.wav', '*.mp3', '*.m4a']
    audio_files = []
    
    for ext in audio_extensions:
        audio_files.extend(temp_folder.glob(ext))
    
    return sorted(audio_files)

def display_batch_info(audio_files: List[Path]):
    """Display information about the batch of files"""
    if not audio_files:
        print("❌ No audio files found in temp folder!")
        return False
    
    total_size_mb = sum(f.stat().st_size for f in audio_files) / (1024 * 1024)
    
    print(f"📊 BATCH PROCESSING INFO:")
    print(f"   Files to process: {len(audio_files)}")
    print(f"   Total size: {total_size_mb:.1f} MB")
    print(f"\n📁 Files found:")
    
    for i, file_path in enumerate(audio_files, 1):
        size_mb = file_path.stat().st_size / (1024 * 1024)
        print(f"   {i}. {file_path.name} ({size_mb:.1f} MB)")
    
    return True

def cleanup_processed_files(processed_files: List[Path]):
    """Delete all successfully processed files"""
    print(f"\n🗑️ CLEANUP: Deleting {len(processed_files)} processed files...")
    deleted_count = 0
    
    for file_path in processed_files:
        try:
            file_path.unlink()  # Delete file
            print(f"   ✅ Deleted: {file_path.name}")
            deleted_count += 1
        except Exception as e:
            print(f"   ❌ Failed to delete {file_path.name}: {e}")
    
    print(f"🗑️ Cleanup complete: {deleted_count}/{len(processed_files)} files deleted")

# Discover files in temp folder
temp_folder = project_root / 'data' / 'temp'
temp_folder.mkdir(parents=True, exist_ok=True)  # Ensure folder exists

audio_files = find_audio_files(temp_folder)
files_available = display_batch_info(audio_files)

if files_available:
    print(f"\n🚀 Ready to process {len(audio_files)} files!")
else:
    print("\n💡 TIP: Add .wav files to data/temp/ folder for testing")

📊 BATCH PROCESSING INFO:
   Files to process: 1
   Total size: 27.6 MB

📁 Files found:
   1. blog_record_hugo_droneflytech.wav (27.6 MB)

🚀 Ready to process 1 files!


In [8]:
# Batch Processing Function (Updated with Full Insights Display)
def process_audio_batch(audio_files: List[Path], pipeline) -> dict:
    """Process all audio files in batch with detailed insights display"""
    
    if not audio_files:
        print("❌ No files to process")
        return {"processed": [], "failed": [], "total": 0}
    
    print(f"\n🚀 STARTING BATCH PROCESSING - {len(audio_files)} files")
    print("=" * 60)
    
    processed_files = []
    failed_files = []
    results = []
    
    for i, file_path in enumerate(audio_files, 1):
        print(f"\n📂 Processing {i}/{len(audio_files)}: {file_path.name}")
        print("-" * 40)
        
        # Create initial state
        initial_state = {
            "file_path": str(file_path),
            "filename": file_path.name,
            "transcript_text": None,
            "conversation_id": None,
            "extracted_insights": None,  
            "error": None,
            "status": "processing"
        }
        
        try:
            # Run through pipeline
            result = pipeline.invoke(initial_state)
            
            if result["status"] in ["completed", "insights_extracted"]:
                print(f"✅ SUCCESS: {file_path.name}")
                print(f"   Conversation ID: {result['conversation_id']}")
                print(f"   Transcript preview: {result['transcript_text'][:100]}...")
                
                # FULL INSIGHTS DISPLAY
                if result.get('extracted_insights'):
                    insights = result['extracted_insights']
                    print(f"\n🧠 === EXTRACTED INSIGHTS FOR: {file_path.name} ===")
                    print("=" * 50)
                    
                    # Speakers
                    if insights.speakers:
                        print("👥 SPEAKERS:")
                        for speaker in insights.speakers:
                            print(f"   • Name: {speaker.name or 'Unknown'}")
                            print(f"     Role: {speaker.role or 'Unknown'}")  
                            print(f"     Company: {speaker.company or 'Unknown'}")
                    
                    # Core Values
                    if insights.core_values:
                        print("💎 CORE VALUES:")
                        for value in insights.core_values:
                            print(f"   • {value}")
                    
                    # Priorities
                    if insights.priorities:
                        print("🎯 PRIORITIES:")
                        for priority in insights.priorities:
                            print(f"   • {priority}")
                    
                    # Primary Challenges
                    if insights.primary_challenges:
                        print("🔥 PRIMARY CHALLENGES:")
                        for challenge in insights.primary_challenges:
                            print(f"   • Challenge: {challenge.description}")
                            print(f"     Impact: {challenge.impact}")
                            print(f"     Urgency: {challenge.urgency}")
                    
                    # Secondary Challenges
                    if insights.secondary_challenges:
                        print("⚠️  SECONDARY CHALLENGES:")
                        for challenge in insights.secondary_challenges:
                            print(f"   • Challenge: {challenge.description}")
                            print(f"     Impact: {challenge.impact}")
                            print(f"     Urgency: {challenge.urgency}")
                    
                    # Current Solutions
                    if insights.current_solutions:
                        print("🔧 CURRENT SOLUTIONS:")
                        for solution in insights.current_solutions:
                            print(f"   • Solution: {solution.solution}")
                            print(f"     Satisfaction: {solution.satisfaction_level}")
                            if solution.limitations:
                                print(f"     Limitations: {', '.join(solution.limitations)}")
                    
                    # Psychological Needs
                    if insights.psychological_needs:
                        print("🧘 PSYCHOLOGICAL NEEDS:")
                        for need in insights.psychological_needs:
                            print(f"   • {need.description}")
                            print(f"     Category: {need.need_category}")
                            print(f"     Intensity: {need.intensity}")
                    
                    print("🧠 === END INSIGHTS ===")
                    print("-" * 50)
                
                processed_files.append(file_path)
            else:
                print(f"❌ FAILED: {file_path.name}")
                print(f"   Status: {result.get('status', 'Unknown')}")
                print(f"   Error: {result.get('error', 'Unknown error')}")
                failed_files.append(file_path)
            
            results.append(result)
            
        except Exception as e:
            print(f"❌ PIPELINE ERROR: {file_path.name}")
            print(f"   Exception: {str(e)}")
            failed_files.append(file_path)
            
            results.append({
                **initial_state,
                "error": str(e),
                "status": "pipeline_error"
            })
    
    # Final Summary
    print(f"\n📊 BATCH PROCESSING COMPLETE!")
    print("=" * 60)
    print(f"✅ Successfully processed: {len(processed_files)}")
    print(f"❌ Failed: {len(failed_files)}")
    print(f"📁 Total files: {len(audio_files)}")
    
    if failed_files:
        print(f"\n❌ Failed files:")
        for failed_file in failed_files:
            print(f"   - {failed_file.name}")
    
    return {
        "processed": processed_files,
        "failed": failed_files,
        "total": len(audio_files),
        "results": results
    }

print("✅ Batch processing function ready with full insights display")

✅ Batch processing function ready with full insights display


In [ ]:
# Cell 14: Analyst Agent Node (Scoring)
def analyst_agent_node(state: AudioPipelineState) -> AudioPipelineState:
    """Analyst agent that scores raw blog ideas"""
    
    try:
        print("🔍 Starting blog idea analysis and scoring...")
        
        raw_ideas = state.get('raw_blog_ideas', [])
        if not raw_ideas:
            return {**state, "error": "No raw ideas to analyze", "status": "error"}
        
        conversation_id = state.get('conversation_id')
        strategy_context = load_company_strategy_context()
        
        scored_ideas = []
        
        for i, raw_idea in enumerate(raw_ideas, 1):
            print(f"📊 Scoring idea {i}/{len(raw_ideas)}: {raw_idea.title}")
            
            # Score the idea using your company strategy
            scored_idea = score_blog_idea_with_strategy(raw_idea, conversation_id, strategy_context)
            scored_ideas.append(scored_idea)
            
            print(f"   Score: {scored_idea.total_score}/70 (Usefulness: {scored_idea.usefulness_potential}/10)")
        
        print(f"✅ Scored {len(scored_ideas)} blog ideas")
        return {
            **state,
            "scored_blog_ideas": scored_ideas,  # BlogPostIdeaCreate objects
            "status": "ideas_scored"
        }
        
    except Exception as e:
        print(f"❌ Analyst agent error: {e}")
        return {**state, "error": str(e), "status": "error"}

print("✅ Analyst agent node ready")

In [9]:
# Cell 6: Define LangGraph Nodes
def transcription_node(state: AudioPipelineState) -> AudioPipelineState:
    """Node 1: Transcribe audio file with AssemblyAI"""
    try:
        print(f"🎙️ Transcribing: {state['filename']}")
        
        # Configure transcriber
        transcriber = aai.Transcriber()
        
        # Transcribe the file
        transcript = transcriber.transcribe(state['file_path'])
        
        if transcript.status == aai.TranscriptStatus.error:
            return {
                **state,
                "error": f"AssemblyAI error: {transcript.error}",
                "status": "transcription_failed"
            }
        
        return {
            **state,
            "transcript_text": transcript.text,
            "status": "transcribed"
        }
        
    except Exception as e:
        return {
            **state,
            "error": f"Transcription error: {str(e)}",
            "status": "transcription_failed"
        }

def database_saver_node(state: AudioPipelineState) -> AudioPipelineState:
    """Node 2: Save transcript to database"""
    try:
        print(f"💾 Saving to database: {state['filename']}")
        
        # Create conversation object
        conversation = ConversationCreate(
            title=f"Audio: {state['filename']}",
            raw_text=state['transcript_text'],
            source="transcribed"
        )
        
        # Save to database
        conversation_id = db.create_conversation(conversation)
        
        return {
            **state,
            "conversation_id": conversation_id,
            "status": "completed"
        }
        
    except Exception as e:
        return {
            **state,
            "error": f"Database error: {str(e)}",
            "status": "database_failed"
        }

print("✅ LangGraph nodes defined")

✅ LangGraph nodes defined


In [15]:
# Cell: Build Current Pipeline (3 Nodes)
def build_pipeline():
    """Build the current LangGraph workflow with transcription, save, and insights"""
    workflow = StateGraph(AudioPipelineState)
    
    # Add current nodes
    workflow.add_node("transcribe", transcription_node)
    workflow.add_node("save_to_db", database_saver_node)  
    workflow.add_node("extract_insights", pain_extractor_node)
    
    # Chain them together
    workflow.add_edge("transcribe", "save_to_db")
    workflow.add_edge("save_to_db", "extract_insights")
    
    workflow.set_entry_point("transcribe")
    workflow.set_finish_point("extract_insights")
    
    return workflow.compile()

# Build the pipeline
pipeline = build_pipeline()
print("✅ LangGraph pipeline compiled (3 nodes: transcribe → save_to_db → extract_insights)")

✅ LangGraph pipeline compiled (3 nodes: transcribe → save_to_db → extract_insights)


In [10]:
# Cell: Clean Conversations Table
def clean_conversations_table():
    """Delete all records from conversations table"""
    
    # First show what will be deleted
    conversations = db.get_all_conversations()
    print(f"📊 Found {len(conversations)} conversations to delete:")
    for conv in conversations[:5]:  # Show first 5
        print(f"  - ID {conv.id}: {conv.title}")
    if len(conversations) > 5:
        print(f"  ... and {len(conversations) - 5} more")
    
    # Ask for confirmation
    response = input(f"\n❓ Delete all {len(conversations)} conversations? (y/N): ")
    
    if response.lower() in ['y', 'yes']:
        conn = db.get_connection()
        try:
            cursor = conn.cursor()
            
            # Delete all conversations (this will also delete related blog_post_ideas due to foreign key)
            cursor.execute("DELETE FROM blog_post_ideas")
            cursor.execute("DELETE FROM processing_status") 
            cursor.execute("DELETE FROM conversations")
            conn.commit()
            
            print("✅ All conversations deleted!")
            print("✅ Related blog ideas deleted!")
            print("✅ Processing status cleared!")
            
        finally:
            conn.close()
    else:
        print("❌ Deletion cancelled")

# Run the cleaner
clean_conversations_table()

📊 Found 0 conversations to delete:
❌ Deletion cancelled


In [16]:
# Cell 9: Execute Batch Processing with Cleanup
if files_available:
    print("🎯 Starting batch processing...")
    
    # Process all files
    batch_results = process_audio_batch(audio_files, pipeline)
    
    # Display summary
    print(f"\n📊 BATCH PROCESSING COMPLETE!")
    print("=" * 60)
    print(f"✅ Successfully processed: {len(batch_results['processed'])}")
    print(f"❌ Failed: {len(batch_results['failed'])}")
    print(f"📁 Total files: {batch_results['total']}")
    
    # Show failed files
    if batch_results['failed']:
        print(f"\n❌ Failed files:")
        for file_path in batch_results['failed']:
            print(f"   - {file_path.name}")
    
    # Cleanup successfully processed files
    if batch_results['processed']:
        confirm = input(f"\n🗑️ Delete {len(batch_results['processed'])} processed files? (y/N): ")
        if confirm.lower() in ['y', 'yes']:
            cleanup_processed_files(batch_results['processed'])
        else:
            print("🔧 Files kept in temp folder for inspection")
    
    print("\n🎉 Batch processing complete!")
    
else:
    print("💡 Add audio files to data/temp/ folder and rerun this cell")

🎯 Starting batch processing...

🚀 STARTING BATCH PROCESSING - 1 files

📂 Processing 1/1: blog_record_hugo_droneflytech.wav
----------------------------------------
🎙️ Transcribing: blog_record_hugo_droneflytech.wav
💾 Saving to database: blog_record_hugo_droneflytech.wav
🧠 Starting pain extraction...
✅ Extracted insights: 1 primary challenges, 1 speakers
✅ SUCCESS: blog_record_hugo_droneflytech.wav
   Conversation ID: 9
   Transcript preview: At Drone Fly Tech, one of our primary challenges revolves around tracking and knowing who paid which...

🧠 === EXTRACTED INSIGHTS FOR: blog_record_hugo_droneflytech.wav ===
👥 SPEAKERS:
   • Name: Hugo
     Role: SpeakerRole.CLIENT
     Company: Drone flytech
💎 CORE VALUES:
   • efficiency
   • transparency
   • accuracy
🎯 PRIORITIES:
   • improving financial processes
   • streamlining invoice tracking
   • enhancing financial management
🔥 PRIMARY CHALLENGES:
   • Challenge: Tracking who paid which invoice
     Impact: Creates confusion and delays 

In [11]:
# Cell 7: Setup Anthropic LLM for Insights Extraction (FIXED)
from langchain_anthropic import ChatAnthropic
import json

# Initialize Anthropic with correct model name
anthropic_key = os.getenv('ANTHROPIC_API_KEY')
if not anthropic_key:
    print("⚠️  ANTHROPIC_API_KEY not found in .env file")
    print("Please add: ANTHROPIC_API_KEY=your_key_here")
else:
    llm = ChatAnthropic(
        model="claude-3-5-sonnet-20241022",  # ← Updated model name
        api_key=anthropic_key,
        temperature=0.1
    )
    print("✅ Anthropic LLM initialized with Claude 3.5 Sonnet")

✅ Anthropic LLM initialized with Claude 3.5 Sonnet


In [12]:
## 3. PainExtractor Node Implementation


import openai
import json
from typing import Dict, Any

# System prompt
PAIN_EXTRACTOR_SYSTEM_PROMPT = """
You are a UX researcher and business analyst for BigKids Automation. Your job is listening to transcripts from interviews with users and potential clients. 

You pay special attention to problems that users have regarding how their company is automating, using web apps and AI to save time and move towards a more ethical and sovereign tech infrastructure.

You will be given the transcript of an interview with a user or potential client.

Your task is to extract structured information about:
- Who is speaking and their role
- What this person cares about (values, priorities)
- Their main primary and secondary challenges
- How they are solving problems today
- Are there AI agents that can assist them?
- Their underlying psychological needs (using frameworks like NVC - Non-Violent Communication)

Focus on automation, web apps, AI, time-saving, ethical tech, and sovereign infrastructure themes.

Be thorough but concise. 

IMPORTANT: Only extract information that is explicitly mentioned in the transcript. 
If information is not clearly stated, leave the field empty/null rather than guessing or inferring.
Do not hallucinate or make assumptions about missing information.
"""

In [13]:
def extract_insights_from_transcript(transcript: str) -> ExtractedInsights:
    """Extract structured insights using Anthropic Claude with proper JSON structure"""
    
    prompt = f"""
    Analyze this conversation transcript and extract structured insights in the exact JSON format below:

    Transcript: {transcript}

    Return ONLY valid JSON in this exact structure:
    {{
        "speakers": [
            {{
                "name": "Hugo",
                "role": "client", 
                "company": "Drone flytech"
            }}
        ],
        "core_values": ["efficiency", "transparency"],
        "priorities": ["improving financial processes"],
        "primary_challenges": [
            {{
                "description": "Tracking who paid which invoice",
                "impact": "Creates confusion in financial processes",
                "urgency": "High"
            }}
        ],
        "secondary_challenges": [],
        "current_solutions": [
            {{
                "solution": "Using MoneyOak software",
                "satisfaction_level": "Unsatisfied", 
                "limitations": ["inadequate functionality", "limited visibility"]
            }}
        ],
        "psychological_needs": [
            {{
                "need_category": "security",
                "description": "Confidence in financial operations",
                "intensity": "High"
            }}
        ]
    }}

    Important: 
    - Return ONLY the JSON, no other text
    - Use exact field names as shown
    - For urgency use: "Low", "Medium", or "High"
    - For satisfaction_level use: "Very Satisfied", "Satisfied", "Neutral", "Unsatisfied", "Very Unsatisfied"
    - For intensity use: "Low", "Medium", or "High"
    - For speaker role use: "client" or "interviewer"
    """
    
    try:
        # Use the Claude LLM
        response = llm.invoke(prompt)
        
        # Parse the JSON response
        insights_data = json.loads(response.content)
        
        # Convert to Pydantic model
        return ExtractedInsights(**insights_data)
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error: {e}")
        print(f"📝 Raw response: {response.content[:500]}...")
        raise
    except Exception as e:
        print(f"❌ Error in LLM call: {e}")
        raise
def pain_extractor_node(state: AudioPipelineState) -> AudioPipelineState:
    """
    LangGraph node: Extract structured insights from conversation transcript
    """
    print("🧠 Starting pain extraction...")
    
    try:
        # Extract insights using OpenAI structured output
        insights = extract_insights_from_transcript(state['transcript_text'])
        
        if insights:
            print(f"✅ Extracted insights: {len(insights.primary_challenges)} primary challenges, {len(insights.speakers)} speakers")
            
            return {
                **state,
                "extracted_insights": insights,
                "status": "insights_extracted"
            }
        else:
            return {
                **state,
                "error": "Failed to extract insights from transcript",
                "status": "error"
            }
            
    except Exception as e:
        print(f"❌ Pain extraction failed: {e}")
        return {
            **state,
            "error": f"Pain extraction error: {str(e)}",
            "status": "error"
        }



In [ ]:
# Cell 16: Complete Creative Agent Test
def test_creative_agent_complete():
    """Complete test of creative agent with real data"""
    
    print("🧪 TESTING CREATIVE AGENT")
    print("=" * 50)
    
    # Step 1: Get a conversation that has insights
    conversations = db.get_all_conversations()
    
    if not conversations:
        print("❌ No conversations found. Run the complete pipeline first.")
        return
    
    # Use the most recent conversation
    test_conversation = conversations[0]
    print(f"📝 Using conversation: {test_conversation.title}")
    print(f"   Word count: {test_conversation.word_count}")
    print(f"   Status: {test_conversation.status}")
    
    # Step 2: For now, we need to run insights extraction on this conversation
    print("\n🧠 Extracting insights for creative agent test...")
    
    if test_conversation.raw_text:
        try:
            # Extract insights
            insights = extract_insights_from_transcript(test_conversation.raw_text)
            print("✅ Insights extracted successfully")
            
            # Step 3: Test creative agent
            print("\n🎨 Testing creative agent...")
            
            # Create test state
            test_state = {
                "file_path": "test.wav",
                "filename": "test.wav", 
                "transcript_text": test_conversation.raw_text,
                "conversation_id": test_conversation.id,
                "extracted_insights": insights,
                "raw_blog_ideas": None,
                "status": "insights_extracted"
            }
            
            # Run creative agent node
            result_state = creative_agent_node(test_state)
            
            # Step 4: Display results
            if result_state.get("raw_blog_ideas"):
                ideas = result_state["raw_blog_ideas"]
                print(f"\n🎉 SUCCESS! Generated {len(ideas)} blog ideas:")
                print("=" * 60)
                
                for i, idea in enumerate(ideas, 1):
                    print(f"\n💡 IDEA {i}:")
                    print(f"Title: {idea['title']}")
                    print(f"Description: {idea['description']}")
                    print(f"Target Audience: {idea['target_audience']}")
                    print(f"Content Angle: {idea['content_angle']}")
                    print(f"Business Value: {idea['business_value']}")
                
                print("=" * 60)
                print("✅ Creative agent test COMPLETED successfully!")
                
            else:
                print("❌ Creative agent failed to generate ideas")
                print(f"Error: {result_state.get('error', 'Unknown error')}")
                
        except Exception as e:
            print(f"❌ Test failed: {e}")
            import traceback
            traceback.print_exc()
    
    else:
        print("❌ No transcript text found in conversation")

# Run the complete test
test_creative_agent_complete()